In [29]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [30]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
import random

In [31]:
tf.keras.backend.clear_session()

In [32]:
subject = 'Mato Grosso Do Sul - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [33]:
data = pd.read_csv('2003_mo_model_input_MS.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Mato Grosso do Sul - IDH,Mato Grosso Do Sul - Produção de Cimento (t),Mato Grosso do Sul - Desemprego,Mato Grosso do Sul - value,Mato Grosso do Sul - PIB - Estadual,Mato Grosso do Sul - PIB - Construção Civil,Mato Grosso do Sul - PIB - Per Capita,Mato Grosso do Sul - PIB - Preços de Mercado,Mato Grosso Do Sul - Consumo de Cimento (t)
0,2003-1,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,0.741626,53.166560,8.248824,0.282837,4.562945e+07,2.151917e+06,16.203179,3.957545e+07,33.744
1,2003-2,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,0.741710,53.245714,8.242935,0.284487,4.568706e+07,2.153542e+06,16.206131,3.959155e+07,32.230
2,2003-3,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,0.741794,52.994640,8.237046,0.285379,4.574467e+07,2.155168e+06,16.209084,3.960765e+07,33.559
3,2003-4,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,0.741878,53.019283,8.231157,0.286089,4.580228e+07,2.156794e+06,16.212036,3.962375e+07,34.283
4,2003-5,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,0.741962,53.539593,8.225268,0.286122,4.585990e+07,2.158419e+06,16.214989,3.963985e+07,36.066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72.097125,NaN,0.523445,NaN,NaN,NaN,NaN,103.854
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71.657488,NaN,0.521590,NaN,NaN,NaN,NaN,95.890
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71.966263,NaN,0.518616,NaN,NaN,NaN,NaN,89.679
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72.205635,NaN,0.515040,NaN,NaN,NaN,NaN,99.356


In [34]:
input_data = data.iloc[:split_index + 1,1:-1]
# mean = np.mean(input_data, axis=0)
# stddev =  np.std(input_data, axis=0)
# input_data = ((input_data - mean) /stddev)
scaler=MinMaxScaler()
scaled_data = scaler.fit_transform(input_data)
input_data = pd.DataFrame(scaled_data, columns=input_data.columns)
input_data

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Mato Grosso do Sul - IDH,Mato Grosso Do Sul - Produção de Cimento (t),Mato Grosso do Sul - Desemprego,Mato Grosso do Sul - value,Mato Grosso do Sul - PIB - Estadual,Mato Grosso do Sul - PIB - Construção Civil,Mato Grosso do Sul - PIB - Per Capita,Mato Grosso do Sul - PIB - Preços de Mercado
0,1.000000,1.000000,0.746035,1.000000,0.000000,0.000000,1.000000,0.000000,0.006282,1.000000,0.477447,0.000000,0.000000,0.000000,0.000000
1,0.921240,0.965719,0.580259,0.936603,0.007992,0.011007,0.968938,0.005809,0.009174,0.996881,0.488349,0.005183,0.011372,0.003635,0.004964
2,0.873107,0.927412,0.926926,0.906543,0.015984,0.022015,0.948438,0.011618,0.000000,0.993763,0.494242,0.010366,0.022743,0.007269,0.009928
3,0.851659,0.885588,0.549924,0.858675,0.023976,0.033022,0.928794,0.017427,0.000900,0.990644,0.498929,0.015549,0.034115,0.010904,0.014892
4,0.823277,0.838034,0.807277,0.824839,0.031968,0.044030,0.970421,0.023236,0.019913,0.987526,0.499148,0.020732,0.045487,0.014538,0.019856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.000000,0.030808,0.063780,0.381401,0.987600,0.828698,0.005731,0.897013,0.208644,0.268279,0.704114,0.972829,0.574311,0.949284,0.958386
188,0.029504,0.027720,0.048101,0.466609,0.990700,0.823090,0.037761,0.889390,0.219400,0.264881,0.723823,0.969261,0.574267,0.942941,0.953579
189,0.043125,0.023074,0.016800,0.530936,0.993800,0.817482,0.087044,0.881766,0.249242,0.261483,0.743880,0.965693,0.574223,0.936598,0.948771
190,0.059756,0.017547,0.035186,0.611274,0.996900,0.811875,0.118264,0.874143,0.274423,0.258086,0.766200,0.962125,0.574179,0.930255,0.943963


In [35]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      36.256
1      33.675
2      43.734
3      36.187
4      33.413
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Mato Grosso Do Sul - Consumo de Cimento (t), Length: 240, dtype: float64

In [36]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Mato Grosso do Sul - IDH,Mato Grosso Do Sul - Produção de Cimento (t),Mato Grosso do Sul - Desemprego,Mato Grosso do Sul - value,Mato Grosso do Sul - PIB - Estadual,Mato Grosso do Sul - PIB - Construção Civil,Mato Grosso do Sul - PIB - Per Capita,Mato Grosso do Sul - PIB - Preços de Mercado
0,1.000000,1.000000,0.746035,1.000000,0.000000,0.000000,1.000000,0.000000,0.006282,1.000000,0.477447,0.000000,0.000000,0.000000,0.000000
1,0.921240,0.965719,0.580259,0.936603,0.007992,0.011007,0.968938,0.005809,0.009174,0.996881,0.488349,0.005183,0.011372,0.003635,0.004964
2,0.873107,0.927412,0.926926,0.906543,0.015984,0.022015,0.948438,0.011618,0.000000,0.993763,0.494242,0.010366,0.022743,0.007269,0.009928
3,0.851659,0.885588,0.549924,0.858675,0.023976,0.033022,0.928794,0.017427,0.000900,0.990644,0.498929,0.015549,0.034115,0.010904,0.014892
4,0.823277,0.838034,0.807277,0.824839,0.031968,0.044030,0.970421,0.023236,0.019913,0.987526,0.499148,0.020732,0.045487,0.014538,0.019856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,0.379453,0.024030,0.338208,0.039971,0.890665,0.956993,0.182507,0.999022,0.386787,0.500849,0.783002,0.985350,0.689228,0.997978,0.993930
158,0.332832,0.021531,0.371581,0.027548,0.893949,0.953662,0.168812,0.998043,0.359381,0.486723,0.772134,0.986682,0.680648,0.998162,0.994482
159,0.313954,0.019385,0.365349,0.037038,0.897232,0.950331,0.152959,0.997065,0.343671,0.472598,0.761307,0.988014,0.672068,0.998346,0.995034
160,0.294856,0.017671,0.404993,0.070841,0.900516,0.947000,0.137613,0.996086,0.322492,0.458472,0.749157,0.989345,0.663488,0.998530,0.995586


In [37]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      36.256
1      33.675
2      43.734
3      36.187
4      33.413
        ...  
157    66.462
158    79.514
159    63.281
160    77.205
161    71.168
Name: Mato Grosso Do Sul - Consumo de Cimento (t), Length: 162, dtype: float64

In [38]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [39]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 15)

In [40]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Mato Grosso do Sul - IDH,Mato Grosso Do Sul - Produção de Cimento (t),Mato Grosso do Sul - Desemprego,Mato Grosso do Sul - value,Mato Grosso do Sul - PIB - Estadual,Mato Grosso do Sul - PIB - Construção Civil,Mato Grosso do Sul - PIB - Per Capita,Mato Grosso do Sul - PIB - Preços de Mercado
126,0.612440,0.052718,0.094523,0.201912,0.780557,0.994213,0.372054,0.701981,0.893564,0.286440,0.986985,0.841584,0.879462,0.935264,0.905184
127,0.626227,0.056653,0.200555,0.209545,0.784414,0.995177,0.375438,0.706029,0.886806,0.301972,0.989794,0.848399,0.878471,0.939850,0.910425
128,0.645442,0.060771,0.214993,0.238038,0.788271,0.996142,0.372254,0.710077,0.875713,0.317504,0.991442,0.855215,0.877481,0.944435,0.915665
129,0.673151,0.065105,0.053026,0.249843,0.792128,0.997106,0.369458,0.714125,0.861745,0.333036,0.993426,0.862031,0.876490,0.949021,0.920905
130,0.699949,0.069099,0.016760,0.266770,0.795984,0.998071,0.371547,0.718173,0.846664,0.348567,0.995712,0.868846,0.875499,0.953606,0.926145
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.000000,0.030808,0.063780,0.381401,0.987600,0.828698,0.005731,0.897013,0.208644,0.268279,0.704114,0.972829,0.574311,0.949284,0.958386
188,0.029504,0.027720,0.048101,0.466609,0.990700,0.823090,0.037761,0.889390,0.219400,0.264881,0.723823,0.969261,0.574267,0.942941,0.953579
189,0.043125,0.023074,0.016800,0.530936,0.993800,0.817482,0.087044,0.881766,0.249242,0.261483,0.743880,0.965693,0.574223,0.936598,0.948771
190,0.059756,0.017547,0.035186,0.611274,0.996900,0.811875,0.118264,0.874143,0.274423,0.258086,0.766200,0.962125,0.574179,0.930255,0.943963


In [41]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 15)

In [42]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [43]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train)
#     display(train_val)
#     display(target)
#     display(target_val)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(36, activation='tanh', 
                                 return_sequences=False), 
                                 input_shape=(train_input.shape[1], 
                                              train_input.shape[2])),
#         tf.keras.layers.Dropout(0.5),
#         tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(180, activation='tanh')),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [44]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(50)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [45]:
winner_seed = get_a_good_seed(reshaped_train, 
                              reshaped_target, 
                              reshaped_test, 
                              reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[887907663, 2672485681, 354855556, 3560037689, 1858203172, 2559452113, 4288369711, 1278213822, 3938603978, 840958352, 2889422159, 3998677049, 3297897540, 2840646772, 3217010714, 1577637084, 3376893128, 2659070513, 1095801597, 3569971524, 4084319611, 3543269830, 1741994316, 878017155, 1742008423, 4126329384, 681901668, 2872152472, 1257754804, 543227109, 636369684, 2211363669, 2262085363, 2095046519, 1109751578, 1225287778, 2895419110, 1314489421, 3329018893, 393627209, 1910052003, 1018444230, 2131587613, 3989514151, 119790049, 165729447, 761114428, 3132141878, 1531343282, 1951680124]


Step: 0 ___________________________________________
val_loss: 25.90546417236328
winner_seed: 887907663


Step: 1 ___________________________________________
val_loss: 30.374555587768555


Step: 2 ___________________________________________
val_loss: 18.318714141845703
winner_seed: 354855556


Step: 3 ___________________________________________
val_loss: 25.722291946411133


Step: 4 _______________________

In [46]:
trained_model, history = bidirectional_lstm_model(reshaped_train, 
                                                  reshaped_target, 
                                                  want_verbose=1, 
                                                  seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 2s 116ms/step - loss: 5997.5972 - val_loss: 1003.8007
Epoch 2/10000
4/4 [==============================] - 0s 8ms/step - loss: 793.5501 - val_loss: 586.4545
Epoch 3/10000
4/4 [==============================] - 0s 8ms/step - loss: 637.8634 - val_loss: 580.2051
Epoch 4/10000
4/4 [==============================] - 0s 8ms/step - loss: 660.6461 - val_loss: 472.2147
Epoch 5/10000
4/4 [==============================] - 0s 9ms/step - loss: 793.6857 - val_loss: 557.5496
Epoch 6/10000
4/4 [==============================] - 0s 9ms/step - loss: 570.9778 - val_loss: 711.9332
Epoch 7/10000
4/4 [==============================] - 0s 9ms/step - loss: 571.8191 - val_loss: 395.1489
Epoch 8/10000
4/4 [==============================] - 0s 9ms/step - loss: 439.9718 - val_loss: 561.6583
Epoch 9/10000
4/4 [==============================] - 0s 9ms/step - loss: 506.1776 - val_loss: 340.0744
Epoch 10/10000
4/4 [==============================] - 0s 9ms/step - l

Epoch 80/10000
4/4 [==============================] - 0s 9ms/step - loss: 204.7015 - val_loss: 198.2431
Epoch 81/10000
4/4 [==============================] - 0s 9ms/step - loss: 187.9344 - val_loss: 175.9915
Epoch 82/10000
4/4 [==============================] - 0s 8ms/step - loss: 188.0203 - val_loss: 216.1471
Epoch 83/10000
4/4 [==============================] - 0s 8ms/step - loss: 182.3768 - val_loss: 202.8870
Epoch 84/10000
4/4 [==============================] - 0s 8ms/step - loss: 170.8356 - val_loss: 222.3535
Epoch 85/10000
4/4 [==============================] - 0s 8ms/step - loss: 219.3009 - val_loss: 285.3174
Epoch 86/10000
4/4 [==============================] - 0s 8ms/step - loss: 183.6617 - val_loss: 187.5538
Epoch 87/10000
4/4 [==============================] - 0s 8ms/step - loss: 220.1195 - val_loss: 218.3959
Epoch 88/10000
4/4 [==============================] - 0s 10ms/step - loss: 208.7141 - val_loss: 202.1441
Epoch 89/10000
4/4 [==============================] - 0s 10ms/s

4/4 [==============================] - 0s 8ms/step - loss: 84.0298 - val_loss: 143.9007
Epoch 159/10000
4/4 [==============================] - 0s 8ms/step - loss: 81.1793 - val_loss: 103.3047
Epoch 160/10000
4/4 [==============================] - 0s 8ms/step - loss: 85.8735 - val_loss: 195.4353
Epoch 161/10000
4/4 [==============================] - 0s 8ms/step - loss: 104.3400 - val_loss: 166.7942
Epoch 162/10000
4/4 [==============================] - 0s 8ms/step - loss: 102.6845 - val_loss: 172.1106
Epoch 163/10000
4/4 [==============================] - 0s 8ms/step - loss: 91.3995 - val_loss: 207.9710
Epoch 164/10000
4/4 [==============================] - 0s 8ms/step - loss: 110.8729 - val_loss: 220.2089
Epoch 165/10000
4/4 [==============================] - 0s 8ms/step - loss: 120.9015 - val_loss: 107.9269
Epoch 166/10000
4/4 [==============================] - 0s 8ms/step - loss: 84.1294 - val_loss: 147.7810
Epoch 167/10000
4/4 [==============================] - 0s 9ms/step - loss: 8

4/4 [==============================] - 0s 9ms/step - loss: 77.6202 - val_loss: 119.0581
Epoch 237/10000
4/4 [==============================] - 0s 9ms/step - loss: 84.4546 - val_loss: 113.4925
Epoch 238/10000
4/4 [==============================] - 0s 8ms/step - loss: 83.6640 - val_loss: 133.3869
Epoch 239/10000
4/4 [==============================] - 0s 8ms/step - loss: 94.7200 - val_loss: 123.1247
Epoch 240/10000
4/4 [==============================] - 0s 8ms/step - loss: 77.5113 - val_loss: 194.8235
Epoch 241/10000
4/4 [==============================] - 0s 8ms/step - loss: 85.5396 - val_loss: 105.2907
Epoch 242/10000
4/4 [==============================] - 0s 8ms/step - loss: 72.4053 - val_loss: 120.4634
Epoch 243/10000
4/4 [==============================] - 0s 8ms/step - loss: 70.9575 - val_loss: 187.3167
Epoch 244/10000
4/4 [==============================] - 0s 8ms/step - loss: 86.7913 - val_loss: 183.4214
Epoch 245/10000
4/4 [==============================] - 0s 8ms/step - loss: 119.0

4/4 [==============================] - 0s 8ms/step - loss: 58.1232 - val_loss: 120.4943
Epoch 315/10000
4/4 [==============================] - 0s 10ms/step - loss: 92.2963 - val_loss: 181.3483
Epoch 316/10000
4/4 [==============================] - 0s 8ms/step - loss: 111.1032 - val_loss: 94.3869
Epoch 317/10000
4/4 [==============================] - 0s 8ms/step - loss: 60.5735 - val_loss: 131.5228
Epoch 318/10000
4/4 [==============================] - 0s 8ms/step - loss: 92.1866 - val_loss: 67.7068
Epoch 319/10000
4/4 [==============================] - 0s 9ms/step - loss: 110.8102 - val_loss: 100.0910
Epoch 320/10000
4/4 [==============================] - 0s 9ms/step - loss: 68.7422 - val_loss: 129.4752
Epoch 321/10000
4/4 [==============================] - 0s 9ms/step - loss: 78.0512 - val_loss: 89.0391
Epoch 322/10000
4/4 [==============================] - 0s 9ms/step - loss: 60.6767 - val_loss: 148.8252
Epoch 323/10000
4/4 [==============================] - 0s 9ms/step - loss: 109.3

...

Epoch 1497/10000
4/4 [==============================] - 0s 9ms/step - loss: 10.0745 - val_loss: 33.3551
Epoch 1498/10000
4/4 [==============================] - 0s 9ms/step - loss: 11.7107 - val_loss: 31.0766
Epoch 1499/10000
4/4 [==============================] - 0s 10ms/step - loss: 12.1095 - val_loss: 24.7989
Epoch 1500/10000
4/4 [==============================] - 0s 9ms/step - loss: 10.7990 - val_loss: 19.7088
Epoch 1501/10000
4/4 [==============================] - 0s 8ms/step - loss: 14.2775 - val_loss: 38.2440
Epoch 1502/10000
4/4 [==============================] - 0s 10ms/step - loss: 12.0635 - val_loss: 37.4611
Epoch 1503/10000
4/4 [==============================] - 0s 9ms/step - loss: 10.7424 - val_loss: 27.8958
Epoch 1504/10000
4/4 [==============================] - 0s 8ms/step - loss: 11.2348 - val_loss: 31.5869
Epoch 1505/10000
4/4 [==============================] - 0s 7ms/step - loss: 10.2359 - val_loss: 25.1955
Epoch 1506/10000
4/4 [==============================] - 0s 8ms

Epoch 1576/10000
4/4 [==============================] - 0s 7ms/step - loss: 44.0590 - val_loss: 88.7719
Epoch 1577/10000
4/4 [==============================] - 0s 7ms/step - loss: 54.1465 - val_loss: 51.4603
Epoch 1578/10000
4/4 [==============================] - 0s 7ms/step - loss: 37.8621 - val_loss: 93.2423
Epoch 1579/10000
4/4 [==============================] - 0s 7ms/step - loss: 85.7406 - val_loss: 62.7203
Epoch 1580/10000
4/4 [==============================] - 0s 7ms/step - loss: 42.1142 - val_loss: 46.4467
Epoch 1581/10000
4/4 [==============================] - 0s 8ms/step - loss: 37.6510 - val_loss: 100.6078
Epoch 1582/10000
4/4 [==============================] - 0s 7ms/step - loss: 67.0365 - val_loss: 64.3607
Epoch 1583/10000
4/4 [==============================] - 0s 8ms/step - loss: 88.4045 - val_loss: 63.4638
Epoch 1584/10000
4/4 [==============================] - 0s 8ms/step - loss: 35.7487 - val_loss: 40.7412
Epoch 1585/10000
4/4 [==============================] - 0s 7ms/

4/4 [==============================] - 0s 9ms/step - loss: 20.0517 - val_loss: 31.4789
Epoch 1655/10000
4/4 [==============================] - 0s 9ms/step - loss: 19.8105 - val_loss: 20.2537
Epoch 1656/10000
4/4 [==============================] - 0s 9ms/step - loss: 16.3548 - val_loss: 21.0812
Epoch 1657/10000
4/4 [==============================] - 0s 9ms/step - loss: 16.2852 - val_loss: 26.9049
Epoch 1658/10000
4/4 [==============================] - 0s 9ms/step - loss: 22.8645 - val_loss: 20.9317
Epoch 1659/10000
4/4 [==============================] - 0s 9ms/step - loss: 26.0378 - val_loss: 25.2260
Epoch 1660/10000
4/4 [==============================] - 0s 9ms/step - loss: 25.0607 - val_loss: 39.9987
Epoch 1661/10000
4/4 [==============================] - 0s 8ms/step - loss: 23.8956 - val_loss: 21.6574
Epoch 1662/10000
4/4 [==============================] - 0s 8ms/step - loss: 13.7044 - val_loss: 25.0867
Epoch 1663/10000
4/4 [==============================] - 0s 8ms/step - loss: 20.70

Epoch 1733/10000
4/4 [==============================] - 0s 8ms/step - loss: 15.7800 - val_loss: 25.2255
Epoch 1734/10000
4/4 [==============================] - 0s 9ms/step - loss: 12.1517 - val_loss: 28.0297
Epoch 1735/10000
4/4 [==============================] - 0s 8ms/step - loss: 13.7339 - val_loss: 25.7580
Epoch 1736/10000
4/4 [==============================] - 0s 10ms/step - loss: 11.7746 - val_loss: 27.8357
Epoch 1737/10000
4/4 [==============================] - 0s 10ms/step - loss: 12.3272 - val_loss: 28.0215
Epoch 1738/10000
4/4 [==============================] - 0s 9ms/step - loss: 12.0844 - val_loss: 20.4496
Epoch 1739/10000
4/4 [==============================] - 0s 9ms/step - loss: 13.5566 - val_loss: 21.2986
Epoch 1740/10000
4/4 [==============================] - 0s 8ms/step - loss: 15.0179 - val_loss: 22.6898
Epoch 1741/10000
4/4 [==============================] - 0s 9ms/step - loss: 11.8838 - val_loss: 24.5569
Epoch 1742/10000
4/4 [==============================] - 0s 9ms

4/4 [==============================] - 0s 8ms/step - loss: 9.6547 - val_loss: 23.1080
Epoch 1812/10000
4/4 [==============================] - 0s 8ms/step - loss: 15.7181 - val_loss: 20.6954
Epoch 1813/10000
4/4 [==============================] - 0s 8ms/step - loss: 12.1950 - val_loss: 27.7688
Epoch 1814/10000
4/4 [==============================] - 0s 8ms/step - loss: 11.1436 - val_loss: 26.8647
Epoch 1815/10000
4/4 [==============================] - 0s 8ms/step - loss: 10.5052 - val_loss: 29.2409
Epoch 1816/10000
4/4 [==============================] - 0s 8ms/step - loss: 10.3467 - val_loss: 29.5035
Epoch 1817/10000
4/4 [==============================] - 0s 8ms/step - loss: 10.6654 - val_loss: 23.8889
Epoch 1818/10000
4/4 [==============================] - 0s 8ms/step - loss: 10.4689 - val_loss: 20.4026
Epoch 1819/10000
4/4 [==============================] - 0s 8ms/step - loss: 13.8365 - val_loss: 29.4535
Epoch 1820/10000
4/4 [==============================] - 0s 8ms/step - loss: 12.312

In [47]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [48]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 14ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,73.175072,72.903343,72.68119,72.412506,72.251068,71.833694,71.665573,71.797752,72.035889,72.184235,72.834488,74.48394,74.952011,76.287056,76.6231,76.087677,75.718651,75.653084,77.357162,77.386803,76.927628,77.290253,77.103073,76.926094,76.853439,76.864555,77.48143,79.061188,81.079948,84.069916
Target,80.517,81.348,76.509,71.851,71.187,58.02,63.194,60.58,69.448,74.472,63.776,81.732,83.858,87.863,69.723,72.765,77.937,64.332,77.048,65.66,65.445,78.095,80.998,69.415,87.346,82.888,79.287,81.855,77.495,56.612
Error,7.341927,8.444656,3.827812,0.561508,1.064072,13.813694,8.471573,11.217751,2.587891,2.287766,9.058487,7.248062,8.905991,11.575943,6.900101,3.322678,2.218346,11.321083,0.309166,11.726799,11.482628,0.804749,3.894928,7.511093,10.492561,6.023445,1.805573,2.793816,3.584946,27.457916


In [49]:
display(mae)
display(mape)

6.935232

0.10054893

In [50]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [51]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

NameError: name 'test_target' is not defined